# Multiclass Image Classification using CNN and transfer learning on Weather images.

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
import random
from keras.models import Model
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import pathlib
import numpy as np
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import os
import cv2
import random
from shutil import copyfile
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.utils import load_img

ImportError: cannot import name 'img_to_array' from 'tensorflow.keras.utils' (C:\Users\Daksh Kapoor\anaconda3\lib\site-packages\tensorflow\keras\utils\__init__.py)

In [ ]:
pip install opencv

In [ ]:
# Get all the paths
data_dir_list = os.listdir("C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Multi-class Weather Dataset-group1")
print(data_dir_list)
path, dirs, files = next(os.walk("C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Multi-class Weather Dataset-group1"))
file_count = len(files)
# print(file_count)

In [ ]:
# Making new base directory, for creating training and test set
original_dataset_dir = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Multi-class Weather Dataset-group1'
base_dir = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/'
os.mkdir(base_dir)

In [ ]:
#create two folders (train and test)
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'test')
os.mkdir(validation_dir)

#Under train folder create four folders 
# (Cloud, Rain, Shine, Sunrise)

train_cloud_dir = os.path.join(train_dir, 'cloudy')
os.mkdir(train_cloud_dir)

train_rainy_dir = os.path.join(train_dir, 'rainy')
os.mkdir(train_rainy_dir)

train_shine_dir = os.path.join(train_dir, 'shine')
os.mkdir(train_shine_dir)

train_sunrise_dir = os.path.join(train_dir, 'sunrise')
os.mkdir(train_sunrise_dir)

#Under test folder create four folders 
# (Cloud, Rain, Shine, Sunrise)

validation_cloud_dir = os.path.join(validation_dir, 'cloudy')
os.mkdir(validation_cloud_dir)

validation_rainy_dir = os.path.join(validation_dir, 'rainy')
os.mkdir(validation_rainy_dir)

validation_shine_dir = os.path.join(validation_dir, 'shine')
os.mkdir(validation_shine_dir)

validation_sunrise_dir = os.path.join(validation_dir, 'sunrise')
os.mkdir(validation_sunrise_dir)



In [ ]:
#This function has been created in order to copy the images from our original data file into the new
# training and test folders we have created, we are giving 4 arguments to the function, source

def split_data(SOURCE, TRAINING, TEST, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + '/' + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    test_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    test_set = shuffled_set[training_length:]

    for filename in training_set:
        this_file = SOURCE + '/' + filename
        destination = TRAINING + '/' +filename
        copyfile(this_file, destination)

    for filename in test_set:
        this_file =  SOURCE + '/' + filename
        destination = TEST + '/' + filename
        copyfile(this_file, destination)

In [ ]:
# The code below has been used 4 times, to create the training and testing data for the 4 seasons.

CLOUDY_SOURCE_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Multi-class Weather Dataset-group1/Cloudy'
TRAINING_CLOUDY_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/train/cloudy'
TEST_CLOUDY_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/test/cloudy'

RAINY_SOURCE_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Multi-class Weather Dataset-group1/Rain'
TRAINING_RAINY_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/train/rainy'
TEST_RAINY_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/test/rainy'

SHINE_SOURCE_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Multi-class Weather Dataset-group1/Shine'
TRAINING_SHINE_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/train/shine'
TEST_SHINE_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/test/shine'

SUNRISE_SOURCE_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Multi-class Weather Dataset-group1/Sunrise'
TRAINING_SUNRISE_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/train/sunrise'
TEST_SUNRISE_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/test/sunrise'

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Exploratory Data Analysis

In [ ]:
TRAINING_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/train'
TEST_DIR = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/test'

In [ ]:
import os
import random
from shutil import copyfile

split_size = .85

split_data(CLOUDY_SOURCE_DIR, TRAINING_CLOUDY_DIR, TEST_CLOUDY_DIR, split_size)
split_data(RAINY_SOURCE_DIR, TRAINING_RAINY_DIR, TEST_RAINY_DIR, split_size)
split_data(SHINE_SOURCE_DIR, TRAINING_SHINE_DIR, TEST_SHINE_DIR, split_size)
split_data(SUNRISE_SOURCE_DIR, TRAINING_SUNRISE_DIR, TEST_SUNRISE_DIR, split_size)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import pathlib

image_folder = ['cloudy','rainy', 'shine', 'sunrise']
nimgs = {}
for i in image_folder:
    nimages = len(os.listdir('C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/train/'+i+'/'))
    nimgs[i]=nimages
plt.figure(figsize=(9, 6))
plt.bar(range(len(nimgs)), list(nimgs.values()), align='center')
plt.xticks(range(len(nimgs)), list(nimgs.keys()))
plt.title('Distribution of different classes in Training Dataset')
plt.show()

In [ ]:
for i in ['cloudy', 'rainy', 'shine', 'sunrise']:
    print('Training {} images are: '.format(i)+str(len(os.listdir('C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/train/'+i+'/'))))

In [ ]:
image_folder = ['cloudy','rainy', 'shine', 'sunrise']
nimgs = {}
for i in image_folder:
    nimages = len(os.listdir('C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/test/'+i+'/'))
    nimgs[i]=nimages
plt.figure(figsize=(9, 6))
plt.bar(range(len(nimgs)), list(nimgs.values()), align='center')
plt.xticks(range(len(nimgs)), list(nimgs.keys()))
plt.title('Distribution of different classes in Validation Dataset')
plt.show()

In [ ]:
for i in ['cloudy', 'rainy', 'shine', 'sunrise']:
    print('Test {} images are: '.format(i)+str(len(os.listdir('C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/test/'+i+'/'))))

In [ ]:
img_width=224; img_height=224 #We have to change the input and the output layers, as we are retaining the weights
# of only hidden or convilutional layers, but since our input image size and number of output classes
# in our case, change, we have to set them accordingly
batch_size=1

# Using VGG16 and doing Data Augmentation.

In [ ]:
IMAGE_SIZE = [224, 224]
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:

train_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1/255.0,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1/255.0)


test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              target_size=(img_height, img_width)
                                                             )

In [ ]:
for layer in vgg16.layers:
  layer.trainable = False

In [ ]:
folders = glob('C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/train*')
print(len(folders))

In [ ]:
x = Flatten()(vgg16.output)
prediction = Dense(4, activation='softmax')(x)
model = Model(inputs=vgg16.input, outputs=prediction)
model.summary()

In [ ]:
from keras import optimizers
adam = optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [ ]:
#Give dataset path
train_path = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/train'
test_path = 'C:/Users/Daksh Kapoor/Desktop/DLAI_FInal_Project/Multi-class Weather Dataset-group1/Weather_image_data/test'

In [ ]:
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

In [ ]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint



checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=2, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model_history=model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=2)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('CNN Model accuracy values')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
acc=model_history.history['accuracy']
val_acc=model_history.history['val_accuracy']
loss=model_history.history['loss']
val_loss=model_history.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(14,7))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Testing Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Graph for loss
fig2 = plt.figure(figsize=(14,7))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Testing Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')

# Using VGG19

In [ ]:
vgg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in vgg19.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(vgg19.output)
prediction = Dense(4, activation='softmax')(x)
model1 = Model(inputs=vgg19.input, outputs=prediction)
model1.summary()

In [ ]:
from keras import optimizers
adam = optimizers.Adam()
model1.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [ ]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint



checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=2, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model1_history=model1.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=2)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
plt.plot(model1_history.history['accuracy'])
plt.plot(model1_history.history['val_accuracy'])
plt.title('CNN Model accuracy values')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#loss
plt.plot(model1_history.history['loss'], label = 'train loss')
plt.plot(model1_history.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
acc=model_history.history['accuracy']
val_acc=model_history.history['val_accuracy']
loss=model_history.history['loss']
val_loss=model_history.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(14,7))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Testing Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Graph for loss
fig2 = plt.figure(figsize=(14,7))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Testing Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')

# Using Resnet

In [ ]:
res50 = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in res50.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(res50.output)
prediction = Dense(4, activation='softmax')(x)
model2 = Model(inputs=res50.input, outputs=prediction)
model2.summary()

In [ ]:
from keras import optimizers
adam = optimizers.Adam()
model2.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [ ]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint



checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=2, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model2_history=model2.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=2)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
acc=model2_history.history['accuracy']
val_acc=model2_history.history['val_accuracy']
loss=model2_history.history['loss']
val_loss=model2_history.history['val_loss']


#loss
plt.plot(model2_history.history['loss'], label = 'train loss')
plt.plot(model2_history.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
#accuracies
plt.plot(model2_history.history['accuracy'], label = 'train accuracy')
plt.plot(model2_history.history['val_accuracy'], label = 'val accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_accuracy')

# Using Xception

In [ ]:
xception = Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in xception.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(xception.output)
prediction = Dense(4, activation='softmax')(x)
model3 = Model(inputs=xception.input, outputs=prediction)
model3.summary()

In [ ]:
from keras import optimizers
adam = optimizers.Adam()
model3.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [ ]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint



checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=2, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model3_history=model2.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=2)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
plt.plot(model3_history.history['accuracy'])
plt.plot(model3_history.history['val_accuracy'])
plt.title('CNN Model accuracy values')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#loss
plt.plot(model3_history.history['loss'], label = 'train loss')
plt.plot(model3_history.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')